In [33]:
from bs4 import BeautifulSoup
import requests
import json
from urllib.parse import urlparse
import pandas as pd
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('data.csv')
df

Saving data.csv to data (1).csv


,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1
...,...,...,...,...
4004,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0
4005,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0
4006,https://www.activistpost.com/2017/09/ron-paul-...,"Ron Paul on Trump, Anarchism & the AltRight",NaN,0
4007,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1


In [34]:
for i in range(4009):
    if('\n\n' in df['URLs'][i]):
        df['URLs'][i] = df['URLs'][i][0:-2]
    if('\r\n' in df['URLs'][i]):
        df['URLs'][i] = df['URLs'][i][0:-2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
def action_parser(link):
    sum = 0
    req = requests.get(link).text
    soup = BeautifulSoup(req, "lxml")
    head = soup.find('head')
    zagolovok = head.find('title').text
    textp = soup.find_all('div', attrs={'class': "article__text"})
    if (len(textp)) == 0:
        textp = soup.find_all('span', attrs={'class': 'intro'})
    if (len(textp)) == 0:
        text = soup.find_all('span', attrs={'class': 'intro'})
    if (len(textp)) == 0:
        textp = soup.find_all('p')

    data = soup.find_all('time')

    if len(data) == 0:
        data = soup.find_all('span', attrs={'class': 'article__info-date-modified'})
    if len(data) == 0:
        data = soup.find_all('div', attrs={'class': 'article__info-date-modified'})

    if len(data) == 0:
      data = soup.find_all('span', attrs={'class': 'DateLine__date___12trWy'})
    if len(data) == 0:
      data = soup.find_all('span', attrs={'class': "entry-meta-date updated"})

    if len(data) == 0:
      data = soup.find_all('div', attrs={'class': "post_date"}) 
           
    if(len(data) == 0):
        return 'non'

    # print(zagolovok)

    data1 = data[0].text

    # for i in range(len(textp)):
    #     print(textp[i].text)

    # if len(data) != 0:
    #     for i in range(len(data)):
    #         print(data[i].text)
    # else:
    #     print("дата публикации на новстоном ресусре не указана")

    # print(data.text)

    domain = urlparse(link).netloc
    if (int(domain.find("www.")) != -1):
        domain = domain[4:]

    url = link.split("//")[-1].split("/")[0].split('?')[0]
    show = "https://input.payapi.io/v1/api/fraud/domain/age/" + domain

    r = requests.get(show)
    soupp = BeautifulSoup(r.text, "lxml")

    if r.status_code == 200:

        age = soupp.find('body').text
        str = int(age.find("Reg"))
        end = int(age.find("}"))
        # print(age[str:(end - 1)])

        data = r.text
        jsonToPython = json.loads(data)

        list = [zagolovok, data1, age[str:(end - 1)]]
        # return list

In [54]:
print(df['URLs'][18])
df = df.drop(18)
df = df.reset_index()
print(df['URLs'][18])


http://beforeitsnews.com/u-s-politics/2017/10/trump-bemoans-little-appreciation-as-san-juan-mayor-begs-for-water-2537595.html
http://beforeitsnews.com/u-s-politics/2017/10/in-meeting-with-military-trump-talks-of-calm-before-the-storm-2537421.html
